- Agnet
    - static agent
    - dynamic agent
- Tools
    - ToolNode
- Prompt
    - string
    - SystemMessage
    - Callable
- Structured Output
- Memory
    - Messagte State
- Pre-model hook
- Post-model hook
- Streaming

### Agent
에이전트는 언어 모델과 도구를 결합하여 작업에 대해 추론하고, 어떤 도구를 사용할지 결정하고, 반복적으로 솔루션을 향애 노력하는 시스템을 만듭니다.

`create_agent() 논문  'ReAct: Synergizing Reasoning and Acting in Language Models'를 기반으로 한 프로덕션에 바로 적용 가능한 ReAct(추론 + 행동) 에이전트 구현을 제공합니다.

---

ReAct 
`thought` 단계: 에이전트의 행동을 여러 `action`, `observation` 단계의 교차로 구성
모델은 추론을 작성하고, 도구를 선택하고, 도구 결과를 확인한 후 이를 반복.

에이전트는 가설을 세우고(`thought`), 도구를 사용하여 검증하고 (`action`), 피드백을 기반으로 계획을 업데이트(`observation`) 함.

ReAct 루프는 정지 조건(모델이 최종 답변을 내보내거나 최대 반복 횟수 제한에 도달할 때)에 도달할 때까지 실행.

## 정적 모델 (Static model)
생성할 때 한번만 구성되며 실행 과정 내내 변경되지 않음.

In [ ]:
from langchain.agents import create_agent

tools = []
agent = create_agent(
    "openai:gpt-5",
    tools=tools
)

`runtime` 에이전트의 실행 환경. 에이전트 실행 기간 내내 유지되는 변경 불가능한 구성과 상황적 데이터(예: 사용자 ID, 세션 세부 정보 또는 애플리케이션별 구성)을 포함.

`state`: 메시지, 사용자 정의 필드, 처리 중에 추적하고 잠재적으로 수정해야 하는 모든 정보(예: 사용자 기본 설정 또는 도구 사용 통계)를 포함하여 에이전트 실행을 통해 흐르는 데이터.

## 동적 모델(Dynamic model)
현재 상태와 컨텍스트를 기반으로 런타임에 선택. 이를 통해 정교한 라우팅 로직과 비용 최적화가 가능.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langgraph.runtime import Runtime

def select_model(state: AgentState, runtime: Runtime) -> ChatOpenAI:
    """Choose model based on conversation complexity."""
    messages = state["messages"]
    message_count = len(messages)

    if message_count < 10:
        return ChatOpenAI(model="gpt-4.1-mini").bind_tools(tools)
    else:
        return ChatOpenAI(model="gpt-5").bind_tools(tools) # Better model for longer conversations

agent = create_agent(select_model, tools=tools)

## 도구 (Tools)
에이전트에게 조취를 취할 수 있는 능력을 제공. 에이전트는 다음과 같은 기능을 통해 단순한 모델 기반 도구 바인딩을 넘어섬.
- 단일 프롬프트에 의해 트리거되는 시퀀스의 여러 도구 호출
- 적절한 경우 병렬 도구 호출
- 결과에 따른 동적 도구 선택
- 도구 재시도 논리 및 오류 처리
- 도구 호출에 따른 상태 지속성

case 1. 도구 목록을 에이전트에 전달하면 ToolNode가 생성.

In [ ]:
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="openai:gpt-4o-mini",
    temperature=0
)
@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def calculate(expression: str) -> str:
    """Perform calculations."""
    return str(eval(expression))

agent = create_agent(model, tools=[search, calculate])

case 2. 구성된 ToolNode 전달

In [ ]:
from langchain.agents import ToolNode

tool_node = ToolNode(
    tools=[search, calculate],
    handle_tool_errors="Please check your input and try again."
)
agent = create_agent(model, tools=tool_node)
result = agent.invoke({"messages": [...]})

# 오류가 발생하면 사용자 지정 오류 메시지와 함께 모델에 반환
# result["messages"]
# [
#     ...
#     ToolMessage(content="Please check your input and try again.", tool_call_id="..."),
#     ...
# ]

## 프롬프트 (Prompt)
제공 방식

In [ ]:
from langchain_core.messages import SystemMessage
# String 제공
agent = create_agent(
    model,
    tools,
    prompt="You are a helpful assistant. Be concise and accurate."
)
# SystemMessage 제공
agent = create_agent(
    model,
    tools,
    prompt=SystemMessage(content="You are a research assistant. Cite your sources.")
)
# Callable
def dynamic_prompt(state):
    user_type = state.get("user_type", "standard")
    system_msg = SystemMessage(
        content="Provide detailed technical responses."
            if user_type == "expert"
            else "Provide simple, clear explanations."
    )
    return [system_msg] + state["messages"]
agent = create_agent(model, tools, prompt=dynamic_prompt)

# prompt가 제공되지 않으면 에이전트는 메시지에서 직접 작업을 추론.

## 구조화된 출력 (Structured Output)
    `response_format` 매개변수를 사용하여 이를 수행할 수 있는 방법을 제공.

In [ ]:
from pydantic import BaseModel
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model,
    tools=[...],
    response_format=ContactInfo
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

## 메모리 (Memory)
메시지 상태를 통해 대화 기록을 자동 관리. 사용자 지정 상태 스키마를 사용하여 대화 중 추가 정보를 기억하도록 구성할 수도 있음.
상태(Messagte State)에 저장된 정보는 에이전트의 "단기 메모리"로 생각할 수 있음.

In [ ]:
from typing_extensions import Annotated # 기존 타입 힌트에 부가적인 정보(메타데이터)를 추가
from langgraph.graph.message import add_messages
from langchain.agents import create_agent
from langchain.agents import AgentState

class CustomAgentState(AgentState):
    messages: Annotated[list, add_messages]
    user_preferences: dict

agent = create_agent(
    model,
    tools=tools,
    state_schema=CustomAgentState
)

# The agent can now track additional state beyond messages. 
# This custom state can be accessed and updated throughout the conversation.
result = agent.invoke({
    "messages": [{"role": "user", "content": "I prefer technical explanations"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})

## Pre-model hook
모델이 호출되기 전에 상태를 처리할 수 있는 선택적 노드.
메시지 트리밍, 요약, 컨텍스트 주입 등에 사용

In [ ]:
from langchain_core.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.agents import create_agent
# 컨텍스트 창에 맞게 메시지를 잘라내는 사전 모델 hook 예시
# {
#     # Will UPDATE the `messages` in the state
#     "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES), ...],
#     # Any other state keys that need to be propagated
#     ...
# }
def trim_messages(state):
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]

    if len(messages) <= 3:
        return {"messages": messages}

    first_msg = messages[0]
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    new_messages = [first_msg] + recent_messages
# If you are returning messages in the pre-model hook, 
# you should OVERWRITE the messages key by doing the following:
# {
# "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES), *new_messages]
# ...
# }
    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages # *: Iterable Unpacking
        ]
    }
# my_list = [2, 3, 4]

# # my_list를 언패킹하여 새로운 리스트를 생성
# new_list = [1, *my_list, 5]

# # 위 코드는 아래와 동일하게 동작합니다.
# # new_list = [1, 2, 3, 4, 5]

# print(new_list)
# # 출력: [1, 2, 3, 4, 5]

agent = create_agent(
    model,
    tools=tools,
    pre_model_hook=trim_messages
)

## Post-model hook
모델 후처리, 도구 실행 전에 모델의 응답을 처리할 수 있는 선택적 노드
* 후처리 이후 도구 호출을 하거나, 응답 전송

검증, 가드레인 또는 기타 hook 처리에 사용

In [ ]:
from langchain_core.messages import AIMessage, RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
# 기밀 정보를 걸러내는 포스트 모델 후크의 예:
def validate_response(state):
    """Check model response for policy violations."""
    messages = state["messages"]
    last_message = messages[-1]

    if "confidential" in last_message.content.lower():
        return {
            "messages": [
                RemoveMessage(id=REMOVE_ALL_MESSAGES),
                *messages[:-1],
                AIMessage(content="I cannot share confidential information.")
            ]
        }

    return {}

agent = create_agent(
    model,
    tools=tools,
    post_model_hook=validate_response
)

## 스트리밍 (Streaming)
에이전트가 여러 단계를 실행하는 경우 시간이 걸릴 수 있음. 준간 진행 상황을 보여주기 위해
메시지가 발생하는 대로 스트리밍할 수 있음.

In [ ]:
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "Search for AI news and summarize the findings"}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")